# COLX 563 Lab Assignment 2: Semantic Role Labelling
## Assignment Objectives

In this lab, you will be implementing and training a neural network (LSTM) model to predict Semantic Role Labels (SRLs). Components of this lab include:

1. Building a pytorch.utils.data.Dataset
2. Implementing a neural network model for SRLabeling
3. Evaluating your network
4. Competing in a Kaggle competition

If you do not have access to a GPU locally you may want to run this on Google CoLab with a GPU backend. Our dataset is not huge, nor is our model, so CPU training is doable (shouldn't take more than about 30 minutes), but training will be *much* faster on a GPU if you are trying different options for the Kaggle competition.


## Getting Started

Run the code below to access relevant modules (you can add to this as needed).

In [1]:
import os, time
import pandas as pd
import codecs
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import f1_score, classification_report
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from bs4 import BeautifulSoup

For this lab, we will be working with the OntoNotes v. 5.0 corpus again. This time, we will be working with data tagged with Semantic Role Labels in the PropBank style using the standard format of CoNLL. Download [the data](https://github.ubc.ca/MDS-CL-2022-23/COLX_563_adv-semantics_students/tree/master/data/Lab2_data.zip) from github, unzip it into a directory outside of your lab repo and change the path below.

In [2]:
ontonotes_path = "data/"

## Tidy Submission
rubric={mechanics:1}

To get the marks for tidy submission:
- Submit the assignment by filling in this Jupyter notebook with your answers embedded
- Be sure to follow the instructions

## Initial Data Processing
In the following cells, we'll be collecting our data and building our data set. **The code has been provided for you. You just need to run the cells.**

First, we'll generate three lists (`train_files`, `dev_files`, and `test_files`) which consist of the paths to all SRL datafiles. 

In [3]:
train_files = ['train/' + filename for filename in os.listdir(ontonotes_path + 'train')]
dev_files = ['dev/' + filename for filename in os.listdir(ontonotes_path + 'dev')]
test_files = ['test/' + filename for filename in os.listdir(ontonotes_path + 'test')]

train_files, dev_files, test_files = sorted(train_files), sorted(dev_files), sorted(test_files)

In [4]:
# Check that we get the correct number of files
assert len(train_files) == 262
print('Success!')

Success!


### Building our dataset

Like with our NER set, we need to generate IOB (**I**nside-**O**utside-**B**eginning) SRL tags for our CoNLL formatted data. But this time, because a sentence can have multiple verbal predicates (with SRL arguments for each predicate), we will potentially create *multiple* IOB tag sequences for a single sentence. 

For your reference, here is an example of CoNLL formatted data. Columns 9 and 10 contain SRL arguments for the verbs *Shooting* and *turn*, respectively. Parentheses like `(ARG2* ... *)` are used to indicate spans for arguments. For example, `ARG2` for the verb *turn* contains the entire span *Into a Funeral in the Gaza Strip*. 

```
ontonotes/wb/a2e/00/a2e_0000   0    0  Celebration    NN  (TOP(S(NP*     -          -    (ARGM-LOC*)   (ARG0*
ontonotes/wb/a2e/00/a2e_0000   0    1     Shooting    NN           *) shoot  shoot.02           (V*)        *)
ontonotes/wb/a2e/00/a2e_0000   0    2        Turns   VBZ        (VP*   turn   turn.02             *       (V*)
ontonotes/wb/a2e/00/a2e_0000   0    3      Wedding    NN        (NP*)    -          -             *    (ARG1*)
ontonotes/wb/a2e/00/a2e_0000   0    4         Into    IN        (PP*     -          -             *    (ARG2*
ontonotes/wb/a2e/00/a2e_0000   0    5            a    DT        (NP*     -          -             *         *
ontonotes/wb/a2e/00/a2e_0000   0    6      Funeral    NN           *)    -          -             *         *
ontonotes/wb/a2e/00/a2e_0000   0    7           in    IN        (PP*     -          -             *         *
ontonotes/wb/a2e/00/a2e_0000   0    8     Southern    JJ        (NP*     -          -             *         *
ontonotes/wb/a2e/00/a2e_0000   0    9         Gaza   NNP           *     -          -             *         *
ontonotes/wb/a2e/00/a2e_0000   0   10        Strip   NNP      *))))))    -          -             *         *)

```
To distinguish between predicates, we'll add a binary indicator feature to **every token**. The value is **1** if the token is the targeted verb and **0**, otherwise. For example, the first SRL column for the sentence above would generate tokens and tags that would look like this (*shooting* is the predicate here as indicated by `('shooting', 1)`):
```
('celebration', 0) : B-ARGM
('shooting', 1) : B-PRED
('turns', 0) : O
('wedding', 0) : O
('into', 0) : O
('a', 0) : O
('funeral', 0) : O
('in', 0) : O
('southern', 0) : O
('gaza', 0) : O
('strip', 0) : O
```
The second SRL column would generate an output that looks like this. The predicate here is *turns*.
```
('celebration', 0) : B-ARG0
('shooting', 0) : I-ARG0
('turns', 1) : B-PRED
('wedding', 0) : B-ARG1
('into', 0) : B-ARG2
('a', 0) : I-ARG2
('funeral', 0) : I-ARG2
('in', 0) : I-ARG2
('southern', 0) : I-ARG2
('gaza', 0) : I-ARG2
('strip', 0) : I-ARG2
```

**Note:** The 'V' tag has been changed to 'PRED,' and all tags 'ARGM(-XYZ)' have been collapsed into a single 'ARGM' type.

The code here reads each sentence as a pandas.DataFrame() and then generates sets of tokens and tags for it.

In [5]:
def df2iob(df):
    '''Generates tokens and tags from a dataframe corresponding to a sentence of the Ontonotes
    corpus tagged for semantic roles. Returns a tuple consisting of a list of lists of 
    (token,is_target_pred) pairs, and lists of lists of IOB tags; the length of these lists will
    correspond to number of predicates in the sentence (potentially zero)'''
    word_tokens = df['Word'].values.tolist()
    all_tags = []
    all_tokens = []
    for column in df:
        if column.startswith("SRL"):
            sent_tags = []
            srl = df[column].values.tolist()    
            in_tag = 0
            for tag in srl:
                # Used to collapse tag classes
                tag = tag.replace('C-', '')
                tag = tag.replace('R-','')
                tag = tag.replace('-DSP','')
                if '(' in tag and ')' in tag:
                    curr_tag = tag[1:-2]
                    curr_tag = 'PRED' if curr_tag == 'V' else curr_tag
                    curr_tag = 'ARGM' if 'ARGM' in curr_tag else curr_tag
                    sent_tags.append('B-' + curr_tag)
                elif '(' in tag:
                    curr_tag = tag[1:-1]
                    curr_tag = 'PRED' if curr_tag == 'V' else curr_tag
                    curr_tag = 'ARGM' if 'ARGM' in curr_tag else curr_tag
                    sent_tags.append('B-' + curr_tag)
                    in_tag = 1
                elif ')' in tag:
                    sent_tags.append('I-' + curr_tag)
                    in_tag = 0
                elif in_tag:
                    sent_tags.append('I-' + curr_tag)
                else:
                    sent_tags.append('O')
            word_token_is_verb = [1 if 'PRED' in tag else 0 for tag in sent_tags]
            sent_token_pairs = [(w.lower(), j) for w,j in zip(word_tokens, word_token_is_verb)]
            all_tokens.append(sent_token_pairs)
            all_tags.append(sent_tags)
    return all_tokens, all_tags


The following function extracts sentences from a file and converts them into Pandas `DataFrame` objects.

In [6]:
column_headers = ['Path', 'SentId', 'WordIdx', 'Word', 'POS', 'Parse', 'Verb', 'VerbSense']

def get_dfs(file):
    '''Gets all dataframes (which correspond to sentences) from a .gold_conll file'''
    dfs = []
    curr_sent = []
    with open(file, encoding='utf=8') as inF:
        for line in inF:
            if line == '\n' and curr_sent:
                num_srl = len(curr_sent[0]) - len(column_headers)
                local_column_headers = column_headers + ['SRL' + str(ii) for ii in range(num_srl)] 
                df = pd.DataFrame(curr_sent, columns=local_column_headers)
                dfs.append(df)
                #Reset for next sentence
                curr_sent = []
            else:
                curr_sent.append(line.strip().split())
    return dfs

Code to make sure things are working properly. Please have a look at `all_tokens` and `all_tags` returned by `df2iob()`.

In [7]:
check_file = ontonotes_path + dev_files[0]
dfs = get_dfs(check_file)
all_tokens, all_tags = df2iob(dfs[3])
assert len(all_tokens) == 3
assert len(all_tags) == 3
assert all_tokens[2] == [('the', 0), ('yugoslav', 0), ('election', 0), ('commission', 0), ('claims', 0), ('he', 0), ('did', 0), ('not', 0), ('win', 1), ('more', 0), ('than', 0), ('50', 0), ('%', 0), ('of', 0), ('the', 0), ('vote', 0), ('.', 0)]
assert all_tags[2] == ['O', 'O', 'O', 'O', 'O', 'B-ARG0', 'O', 'B-ARGM', 'B-PRED', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O']
assert all_tokens[0] == [('the', 0), ('yugoslav', 0), ('election', 0), ('commission', 0), ('claims', 1), ('he', 0), ('did', 0), ('not', 0), ('win', 0), ('more', 0), ('than', 0), ('50', 0), ('%', 0), ('of', 0), ('the', 0), ('vote', 0), ('.', 0)]
assert all_tags[0] == ['B-ARG0', 'I-ARG0', 'I-ARG0', 'I-ARG0', 'B-PRED', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O']
print('Success!')

Success!


In [8]:
all_tokens
all_tags

[['B-ARG0',
  'I-ARG0',
  'I-ARG0',
  'I-ARG0',
  'B-PRED',
  'B-ARG1',
  'I-ARG1',
  'I-ARG1',
  'I-ARG1',
  'I-ARG1',
  'I-ARG1',
  'I-ARG1',
  'I-ARG1',
  'I-ARG1',
  'I-ARG1',
  'I-ARG1',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PRED',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'B-ARG0',
  'O',
  'B-ARGM',
  'B-PRED',
  'B-ARG1',
  'I-ARG1',
  'I-ARG1',
  'I-ARG1',
  'I-ARG1',
  'I-ARG1',
  'I-ARG1',
  'O']]

## Exercise 1: Building a pytorch Dataset

Now we are going to build a `pytorch.utils.data.Dataset` with tensor representations of our data. 

We're not doing this using torchtext. Instead, we'll do it more or less from scratch because that's a good skill to know. For example, you won't be able to use torchtext when handling non-language data. 

### Exercise 1.1
rubric = {accuracy:1}

First, we need to generate our entire dataset. Before we create the pytorch `Dataset`, our input will be word tokens and their binary value indicating if they are the target verb predicate for the current input. The output will be an IOB-SRL tag. 

Using the functions `get_dfs()` and `df2iob()` provided in **Initial Data Processing** above, write a `prepare_SRL_data` function that takes as input a list of .gold_conll files and creates a two python lists `input_tokens` and `output_tags`. Each list should contain an element for every sentense in the input files. 

The lists in `input_tokens` should contain `(token, indicator)` pairs, where `token` is a word token and `indicator` either 0 or 1 as returned by `df2iob()`.

The lists in `output_tags` should contain IOB tags.

**HINT:** Check how to use the functions `get_dfs()` and `df2iob()` in the assert cell above.

In [9]:
def prepare_SRL_data(srl_files):
    '''create IOB-data for all the SRL-tagged Ontonotes files in srl_files. 
    
       Input: list of file names
       Output: (input_tokens, output_tags) 
       
       input_tokens is a list of lists, one list for every sentence in the data. Here 
                    each sentence list consists of (token, is_target_pred) pairs. 
       output_tags  is a list of lists, one list for every sentence in the data. Here 
                    each sentence list consists of IOB tags.
    '''
    input_tokens, output_tags = [], []
    
    for f in srl_files:
        for i in get_dfs(ontonotes_path + f):
            tokens, tags = df2iob(i)
            input_tokens.extend(tokens)
            output_tags.extend(tags)
        
    
    return input_tokens, output_tags

In [10]:
input_tokens, output_tags = prepare_SRL_data(train_files)
assert len(input_tokens) == 22437
assert len(output_tags) == 22437
print('Success!')

Success!


### Exercise 1.2
rubric={accuracy:1, quality:1}

Now we need to pad our dataset so that all sequences (sentences) have the same length. This is needed to allow us to train our SRL model in batches. To do the padding, we first need to find the longest sentence in your dataset (``input_tokens``).

In [11]:
# Finding the longest sentence
LONGEST_SENTENCE = max([len(sent) for sent in input_tokens])
assert LONGEST_SENTENCE == 100

Next, write two functions ``pad_tokens()`` and ``pad_tags()``  that take in, respectively, either the `input_tokens` list or the `output_tags` list created by `prepare_SRL_data` as well as `longest_sentence` and returns the same data with padding for each of the sentences. Padding should be done at the end of each sentence. You can pad with the tuple `(PAD, 0)` for token padding and the string `PAD` for tag padding.

**Note:** It would result in less padding if we separately padded each batch (instead of padding all the sentences in the dataset to the same length), however, this would also complicate our code, so we'll take the simpler approach here.

In [12]:
def pad_tokens(input_tokens, longest_sentence):
    ### Pad tokens list (ie, the X values of the data) 
    ### with <null> to extend the size of the list
    result = []
    for sent in input_tokens:
        padded = []
        padding = [("<null>", 0) for i in range(longest_sentence - len(sent))]
        padded.append(sent + padding)
        result.extend(padded)

    return result

def pad_tags(output_tags, longest_sentence):
    ### Pad tags list (ie, the Y values of the data) 
    ### with <null> to extend the size of the list
    result = []
    for sent in output_tags:
        padded = []
        padding = ["<null>" for i in range(longest_sentence - len(sent))]
        padded.append(sent + padding)
        result.extend(padded)

    return result


In [13]:
padded_input_tokens = pad_tokens(input_tokens[0:1], LONGEST_SENTENCE) 
padded_output_tags = pad_tags(output_tags[0:1], LONGEST_SENTENCE)

assert len(padded_input_tokens[0]) == LONGEST_SENTENCE
assert len(padded_output_tags[0]) == LONGEST_SENTENCE
assert padded_input_tokens[0][-1] == ("<null>", 0)
assert padded_output_tags[0][-1] == "<null>" 
print('Success!')

Success!


### Exercise 1.3
rubric = {accuracy:2}

Now, we need to vectorize our input vocabulary and output tags. The code below creates a number of global variables to help us with this. 

* The `WORD_TO_IDX` dictionary maps words occurring in the training data into unique integers. 
* The `TAG_TO_IDX` dictionary maps *every* tag in our data to a unique integer. 
* The `IDX_TO_TAG` dictionary maps the other way so you can decode tags later.

In [14]:
PAD = "<null>"
UNK = "<unk>"

words = set([word[0] for sent_words in input_tokens for word in sent_words])

# Padding and unknown token
words.add(PAD)
words.add(UNK)

tags = list(set([tag for sent_tags in output_tags for tag in sent_tags]))

# Padding tag
tags.append(PAD)

WORD_TO_IDX = {w:i for i,w in enumerate(words)}
TAG_TO_IDX = {t:i for i,t in enumerate(tags)}
IDX_TO_TAG = tags

Assertions to check your code.

In [15]:
assert len(WORD_TO_IDX) == 8983
assert len(TAG_TO_IDX) == 16

Next, write two functions `converttokens2tensors` and `converttags2tensors` which respectively convert lists of sentences of tokens or tags into lists of `torch.tensor()` objects (one tensor per sentence).  

For tokens, each tensor should be of shape `3 x longest_sentence`. The three rows in the tensor are:
1. The word index numbers for all words in the sentence, which you can get from the map `WORD_TO_IDX`. If a token is not in `WORD_TO_IDX` (i.e. it is an OOV token), you can replace the token with `UNK`.
2. The `is_target_pred` binary features, represented by **integers 1 and 0**. 
3. The third row also represents an indicator feature with value 0 or 1. The value should be 0 if the character is `PAD` and 1 otherwise. 

For tags, each tensor should be a `torch.Tensor()` vector of length `longest_sentence`, corresponding to the integer indexes of the tags. Use `torch.long` as the dtype for both kinds of tensors.

In [45]:
def converttokens2tensors(input_tokens):
    '''
        Convert input_tokens into a tensor
        
        input:  A list of sentences consisting of (token, is_target_pred) pairs.
        output: A list of 2 x sentence_length tensor t
        
        where t[0][i] gives the index number for token i, t[1][i] is either 0 or 1 
        depending on is_target_pred and t[2][i] indicates whether this is a regular
        token or padding token PAD.
    '''
    token_tensors = []
    unk = WORD_TO_IDX["<unk>"]
    for sent in input_tokens:
        words2idx,is_target,indicator = [],[],[]

        for word, tag in sent:
            words2idx.append(WORD_TO_IDX.get(word, unk))
            is_target.append(tag)
            indicator.append(0) if word == PAD else indicator.append(1)

        t =  torch.tensor([words2idx,is_target,indicator], dtype = torch.long)
        token_tensors.append(t)
    
    return token_tensors

def converttags2tensors(output_tags):
    '''
        Convert output_tags into a tensor

        input:  A list of IOB tag sequences.
        output: A list of vectors of index numbers correspinding to IOB tags 
    '''
    tag_tensors = []

    for tags in output_tags:
        # for each sentence: [1..100]               : TAG_TO_IDX, pad; 
        tmp = [TAG_TO_IDX[tag] for tag in tags]

        t = torch.tensor(tmp, dtype = torch.long) 
        tag_tensors.append(t)

    return tag_tensors

In [46]:

check_tokens, check_tags = converttokens2tensors(padded_input_tokens[:1]), converttags2tensors(padded_output_tags[:1])
print(check_tokens)
print("----")
print(check_tags)

[tensor([[3922, 7607, 1620, 8163, 1610, 2363, 8728,  802, 7362, 8728, 2851, 8876,
         3337, 8378, 2851, 8778, 7973, 3140, 5441,  838,  838,  838,  838,  838,
          838,  838,  838,  838,  838,  838,  838,  838,  838,  838,  838,  838,
          838,  838,  838,  838,  838,  838,  838,  838,  838,  838,  838,  838,
          838,  838,  838,  838,  838,  838,  838,  838,  838,  838,  838,  838,
          838,  838,  838,  838,  838,  838,  838,  838,  838,  838,  838,  838,
          838,  838,  838,  838,  838,  838,  838,  838,  838,  838,  838,  838,
          838,  838,  838,  838,  838,  838,  838,  838,  838,  838,  838,  838,
          838,  838,  838,  838],
        [   0,    0,    0,    0,    1,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,

Assertions to check your code.

In [47]:
check_tokens, check_tags = converttokens2tensors(padded_input_tokens[:5]), converttags2tensors(padded_output_tags[:5])
assert check_tokens[0].shape == torch.Size([3, LONGEST_SENTENCE])
assert check_tags[0].shape == torch.Size([LONGEST_SENTENCE])
print('Success!')

Success!


### Final data preparation

Below we provide code that does end-to-end conversion from the CoNLL format to a pytorch dataset object (which will allow us to use a `DataLoader` object). Since we have already done all the preprocessing necessary to have a nicely formated dataset, this is straightforward, and the code is provided. First, we define a pytorch `Dataset`.

In [48]:
class SRL_dataset(Dataset):
    def __init__(self, input_data, output_data):
        self.input_data = input_data
        self.output_data = output_data
        
    def __len__(self):
        return len(self.input_data)
    
    def __getitem__(self, index):
        target = self.output_data[index]
        data_val = self.input_data[index]
        return data_val,target     

The function below loads a set of files as a `SRL_dataset` in the correct format

In [49]:
def prepare_dataset(files):
    '''given a list of Ontonotes SRL, loads them into a pytorch Dataset'''
    input_tokens, output_tags = prepare_SRL_data(files)
    padded_input_tokens, padded_output_tokens = pad_tokens(input_tokens, LONGEST_SENTENCE), pad_tags(output_tags, LONGEST_SENTENCE)
    input_token_tensors, output_tags_tensors = converttokens2tensors(padded_input_tokens), converttags2tensors(padded_output_tokens)
    return SRL_dataset(input_token_tensors, output_tags_tensors)

Assertions to check that everything is working correctly.

In [50]:
batch_size = 64

train_dataset = prepare_dataset(train_files)
assert len(train_dataset) == 22437

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
for train_token_batch, train_tag_batch in train_dataloader:
    assert train_token_batch.shape == torch.Size([batch_size, 3, LONGEST_SENTENCE])
    assert train_tag_batch.shape == torch.Size([batch_size, LONGEST_SENTENCE])
    break
    
print('Success!')

Success!


## Exercise 2: Semantic Role Labeling with a biLSTM

Now we will implement the biLSTM model from Semantic Role Labeling. A visual representation of the model is presented below. You'll notice that the input to the model is a `(token, token_is_target_verb)` tuple like those we just generated.

<img src="data:image/png;base64, ..." alt="Red dot" />



### Exercise 2.1
rubric={accuracy:3}

Define a `biLSTMTagger` model. You should adapt the code from a basic LSTMTagger model [here](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html). You'll need to adapt the `__init__()` function in a couple of ways:
1. You should be able initialize the class as `biLSTMTagger(WORD_TO_IDX, TAG_TO_IDX)`. You can use constants like `EMBEDDING_DIM` below to set all dimensionalities. 
2. Add a new embedding object, namely `self.indicator_embedding`, where we only have two vectors, one for the indicator value 1 and another one for indicator value 0. The embedding dimension should be that same as for the word embeddings, i.e. `EMBEDDING_DIM`.
3. Change the input dimension of `self.lstm` to `2 * EMBEDDING_DIM` and make the LSTM layer bidirectional. Initialize the LSTM with `HIDDEN_DIM` and 1 layer.
4. Change the input dimension of `self.hidden2tag` to `2 * HIDDEN_DIM`.

You also need to change the `forward()` function:
1. Modify the function so that it can take input of dimension `batch_size x 3 x sentence_length` (you may want to immediately transpose this into `3 x sentence_length x batch_size` because this will be compatible with torch LSTM).
2. You'll need to separately generate embeddings for word tokens and indicator features, using the first and second row of the input batch, respectively. You can then concatenate those into a `sentence_length x batch_size x 2*EMBEDDING_DIM` tensor.
3. You should then use the third row in the input batch to generate a vector `sentence_lengths` of size `batch_size` containing the lengths of the sentences in the input batch. The length refers to the number of actual word tokens as opposed to padding symbols.  
4. Pack your embeddings into a torch RNN input batch using [`pack_padded_sequence`](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_padded_sequence.html). You will need to feed in `sentence_lengths` as a parameter.
5. Call `self.lstm` on the result and transform the output back into a regular padded tensor using [`pad_packed_sequence`](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_packed_sequence.html). **Note:** You should set the parameter `total_length` to `LONGEST_SENTENCE` in order to pad all sentences in the dataset to the same length.

Finally, pass the LSTM hidden states (one for every position in the sequence) through `self.hidden2tag` and `log_softmax` and return the result.

In [51]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Here are the embedding and hidden dimensionalities as well as LSTM layer count
EMBEDDING_DIM=75
HIDDEN_DIM=50
LAYERS=1

# your code here
class biLSTMTagger(nn.Module):

    def __init__(self, vocab, tagset):
        super(biLSTMTagger, self).__init__()

        # requires vocab, tagset; 

        self.word_embeddings = nn.Embedding(len(vocab), EMBEDDING_DIM)       # `Embedding` with `len(vocab)` with EMBEDDING_DIM; 
        self.indicator_embedding = nn.Embedding(2, EMBEDDING_DIM)   # another `Embedding` for "indicator_embedding (len(0 or 1)) == 2) with EMBEDDING_DIM ; 
        self.lstm = nn.LSTM(2*EMBEDDING_DIM, HIDDEN_DIM, bidirectional = True)                       # `LSTM` with 2 * EMBEDDING_DIM, HIDDEN_DIM, and bidrectional = True

        self.hidden2tag = nn.Linear(HIDDEN_DIM*2, len(tagset))               # The linear layer that maps from hidden state space to tag space (output = len(tagset))

    def forward(self, batch):
        # 1. Modify the function so that it can take input of dimension `batch_size x 3 x sentence_length` 
        #       (you may want to immediately transpose this into `3 x sentence_length x batch_size` because this will be compatible with torch LSTM).
        # `permute` (0,1,2)  -> (1,2,0) 
        batch = torch.permute(batch, (1,2,0))  # required to be permuted: 3 x sent_len x batch_size

        # 2. You'll need to separately generate embeddings for word tokens and indicator features, 
        #   using the first and second row of the input batch, respectively. 
        #   You can then concatenate those into a `sentence_length x batch_size x 2*EMBEDDING_DIM` tensor.
        embeds_word = self.word_embeddings(batch[0])
        embeds_ind = self.indicator_embedding(batch[1])
        embeds = torch.cat((embeds_word, embeds_ind), dim=2)                        # sent_len x batch_size x 2*EMBEDDING_DIM

        # 3. You should then use the third row in the input batch to generate a vector `sentence_lengths` of size `batch_size` 
        #   containing the lengths of the sentences in the input batch. 
        #   The length refers to the number of actual word tokens as opposed to padding symbols.  
        word_lengths = batch[2].sum(dim=0)                         # word_lengths by summing `batch[2]`
 
        # 4. Pack your embeddings into a torch RNN input batch using `pack_padded_sequence`. 
        # You will need to feed in `sentence_lengths` as a parameter. + `enforce_sorted=False`
        # print(embeds.size())
        # print(word_lengths)
        lstm_input = pack_padded_sequence(embeds, word_lengths, enforce_sorted=False)                      # requires `pack_padded_sequence`
 
        # 5. Call `self.lstm` on the result and transform the output back into a regular padded tensor using `pad_packed_sequence`
        # **Note:** You should set the parameter `total_length` to `LONGEST_SENTENCE` in order to pad all sentences in the dataset to the same length.     
        lstm_out, _ = self.lstm(lstm_input)                       # sent_len x batch_size x 2*HIDDEN_DIM
        lstm_out, _ = pad_packed_sequence(lstm_out, total_length = LONGEST_SENTENCE)                     
        
        tag_space = self.hidden2tag(lstm_out)                            # generated by `hidden2tag`: sent_len x batch_size x tagset_size
        tag_scores = F.log_softmax(tag_space, dim=2)                # then, softmax; 

        return tag_scores

An assertion to check your code:

In [52]:
tagger = biLSTMTagger(WORD_TO_IDX, TAG_TO_IDX)
batch = next(iter(train_dataloader))[0]
assert tagger(batch).size() == (batch.size()[2], batch_size, len(TAG_TO_IDX))

In [53]:
batch_size

64

In [54]:
batch.size()

torch.Size([64, 3, 100])

### Exercise 2.2
rubric={accuracy:1}

Now, we'll need to train our LSTMTagger. Initialize your LSTMTagger and prepare your training data with:

- 20 epochs
- a NLLLoss() loss function
- an SGD optimizer with learning rate 0.1

In [55]:
EPOCHS = 20 
model = biLSTMTagger(WORD_TO_IDX, TAG_TO_IDX)
loss_function = nn.NLLLoss(ignore_index=TAG_TO_IDX[PAD])
optimizer = optim.SGD(model.parameters(), lr=0.1)

### Exercise 2.3
rubric={accuracy:1}

Now train the model. Before training on a GPU, it will be worthwhile to generate a small dataset to make sure the model is training as expected. You should print out a loss at the end of each epoch. Feel free to adapt the training code from other neural network labs you have done in this program, there's nothing particularly special about this one.  If you see anything weird, like the loss increasing, or epochs taking forever on a small dataset, there's probably a bug in your code.  Try to get this part working before moving on to the entire dataset.

Remember that the general NN training process / epoch consists of:
    1. zeroing the gradient
    2. passing your batch through the model
    3. calculating the loss (make sure the output of your model matches the shape of your gold labels!)
    4. back-propagating the loss
    5. taking a step with the optimizer
    
Optionally, you may want to give your model functionality to save / load from a file (especially if you are running on a CPU).

In [56]:
# Generate a small training set containing the 50 first batches
mini_trainset = list(train_dataloader)
print()

for epoch in range(EPOCHS):  
    print(f"Epoch {epoch+1}")
    tot_loss = 0    

    for sent_batch, tag_batch in mini_trainset:

        # `zero_gread()`
        model.zero_grad()
        
        output = model(sent_batch)
        output = output.permute(1,2,0) # batch size x 3 x sentence length
        loss = loss_function(output, tag_batch)
        loss.backward()
        tot_loss += loss
        optimizer.step()
        
    print(f"Avg loss per sentence: {tot_loss/len(mini_trainset)}")


Epoch 1
Avg loss per sentence: 1.2754137516021729
Epoch 2
Avg loss per sentence: 1.060939073562622
Epoch 3
Avg loss per sentence: 0.9845103025436401
Epoch 4
Avg loss per sentence: 0.940952479839325
Epoch 5
Avg loss per sentence: 0.9110361337661743
Epoch 6
Avg loss per sentence: 0.8864666819572449
Epoch 7
Avg loss per sentence: 0.8651724457740784
Epoch 8
Avg loss per sentence: 0.8459624648094177
Epoch 9
Avg loss per sentence: 0.8250940442085266
Epoch 10
Avg loss per sentence: 0.8051414489746094
Epoch 11
Avg loss per sentence: 0.7855508327484131
Epoch 12
Avg loss per sentence: 0.7687463164329529
Epoch 13
Avg loss per sentence: 0.7534292936325073
Epoch 14
Avg loss per sentence: 0.7399144768714905
Epoch 15
Avg loss per sentence: 0.7255632877349854
Epoch 16
Avg loss per sentence: 0.7136132717132568
Epoch 17
Avg loss per sentence: 0.702505886554718
Epoch 18
Avg loss per sentence: 0.6922375559806824
Epoch 19
Avg loss per sentence: 0.6828761100769043
Epoch 20
Avg loss per sentence: 0.67490535

### Exercise 2.4
rubric = {accuracy:2,quality:1}


Now print out a classification report of your model for the `dev_data`. You'll need predict the tag_scores and convert them to text using your `IDX_TO_TAG` dictionary from before. You should skip the padding symbol (i.e. `PAD` tag) in your calculations using the `labels` option for `classification_report`.

Your micro average f1-score should be above 70%.

In [77]:
dev_dataset = prepare_dataset(dev_files)
dev_dataloader = DataLoader(dev_dataset, batch_size=1, shuffle=False)

sys_tags = []
gold_tags = []

# your code here
for sent, tags in dev_dataloader:
    # model
    output = model(sent)
    # argmax of the reuslt of `model`
    tag = torch.argmax(output, dim=2).squeeze(1)
    # accumulate predicted tags
    sys_tags.extend([IDX_TO_TAG[i] for i in tag])
    # accumulate gold tags (`tags`)
    # print("sys:", len(sys_tags))
    # print(len(tags.squeeze(0)))
    # print(tags.squeeze(0))
    gold_tags.extend([IDX_TO_TAG[i] for i in tags.squeeze(0)])
    # print(len(gold_tags))


print(classification_report(sys_tags, gold_tags, labels=list(TAG_TO_IDX.keys())[:-1]))


c:\Users\chris\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\chris\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      B-ARG2       0.30      0.66      0.41       444
      I-ARGM       0.07      0.41      0.12       784
      B-ARG0       0.45      0.76      0.56      1035
      I-ARG0       0.03      0.68      0.07        72
      I-ARG2       0.26      0.66      0.38      1634
      B-ARG3       0.00      0.00      0.00         0
           O       0.97      0.15      0.27    378076
      B-PRED       1.00      0.98      0.99      4056
      B-ARG1       0.55      0.63      0.59      2391
      I-ARG4       0.00      0.00      0.00         0
      I-PRED       0.06      1.00      0.11         4
      I-ARG1       0.52      0.69      0.59      8338
      B-ARGM       0.26      0.52      0.35      1166
      B-ARG4       0.00      0.00      0.00         0
      I-ARG3       0.00      0.00      0.00         0

   micro avg       0.77      0.18      0.30    398000
   macro avg       0.30      0.48      0.30    398000
weighted avg       0.95   

c:\Users\chris\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [78]:
print(len(sys_tags))

398000


In [79]:
print(len(gold_tags))

398000


## Exercise 3: Kaggle Competition 

### Exercise 3.1
rubric={accuracy:2}

There's one small additional challenge to preparing for this week's Kaggle competition: although the test data is in CoNLL format, it is entirely missing the SRL columns, so you won't be able to use  `df2iob` to build your set of tokens with predicate indicators 0/1 (which are also needed in prediction mode). Here's an example of the test data format:

```
ontonotes/wb/a2e/00/a2e_0000   0    0  Celebration    NN  (TOP(S(NP*     -          -
ontonotes/wb/a2e/00/a2e_0000   0    1     Shooting    NN           *) shoot  shoot.02
ontonotes/wb/a2e/00/a2e_0000   0    2        Turns   VBZ        (VP*   turn   turn.02
ontonotes/wb/a2e/00/a2e_0000   0    3      Wedding    NN        (NP*)    -          -
ontonotes/wb/a2e/00/a2e_0000   0    4         Into    IN        (PP*     -          -
ontonotes/wb/a2e/00/a2e_0000   0    5            a    DT        (NP*     -          -
ontonotes/wb/a2e/00/a2e_0000   0    6      Funeral    NN           *)    -          -
ontonotes/wb/a2e/00/a2e_0000   0    7           in    IN        (PP*     -          -
ontonotes/wb/a2e/00/a2e_0000   0    8     Southern    JJ        (NP*     -          -
ontonotes/wb/a2e/00/a2e_0000   0    9         Gaza   NNP           *     -          -
ontonotes/wb/a2e/00/a2e_0000   0   10        Strip   NNP      *))))))    -          -

```

You should create a new version of `prepare_SRL_data` that gets around this problem by using the `VerbSense` column of your dataframe (that's the last column in the table). For each token with a verb sense (`shoot.02` and `turn.02` above), you should generate one list of tokens with indicator features 0/1.

Be sure to create these instances in the order that the verbs appear in the sentences, to keep your format consistent with ours so that the automatic evaulation for the Kaggle competition will work correctly. **Also, remember to lower-case your tokens!** 

You will need to output tags to keep things consistent with the original version of `prepare_SRL_data`, you can just have dummy O tags for everything.

From the table above, you should generate two token lists. First:
```
('celebration', 0)
('shooting', 1)
('turns', 0)
('wedding', 0)
('into', 0)
('a', 0)
('funeral', 0)
('in', 0)
('southern', 0)
('gaza', 0)
('strip', 0)
```
and, second: 
```
('celebration', 0)
('shooting', 0)
('turns', 1)
('wedding', 0)
('into', 0)
('a', 0)
('funeral', 0)
('in', 0)
('southern', 0)
('gaza', 0)
('strip', 0)
```

You also need to generate tag sequences containing all O's.

**Hint:** You can do this using Pandas. It's also fine to read the data using native Python functions.

**Hint:** Depending on your code, you may get empty token and tag lists. You should skip those.

In [80]:
from os.path import join

COLUMN_HEADERS = ['Path', 'SentId', 'WordIdx', 'Word', 'POS', 'Parse', 'Verb', 'VerbSense']

    
def prepare_SRL_data(files):
    token_lists = []
    tag_lists = []
    
    for filename in files:
        with open(join(ontonotes_path,filename)) as f:  
            sentences = f.read().split("\n\n")
            sentences = [[line.split("\t") for line in sent.split("\n")] for sent in sentences if not sent == ""]
            for sent in sentences:
                df = pd.DataFrame(sent, columns=COLUMN_HEADERS)
                indicators = (df["VerbSense"] != "-").values
                words = df["Word"].values.tolist()
                tags = ["O" for j in range(len(words))]
                for i, indicator in enumerate(indicators):
                    if indicator:
                        token_lists.append([(w.lower(), 1 if j == i else 0) for j, w in enumerate(words)])
                        ind = np.zeros(len(words),dtype=int)
                        ind[i] = 1
                        tag_lists.append(tags)
    return token_lists, tag_lists
    

# A test to check that your function works correctly
token_lists, tag_lists = prepare_SRL_data([test_files[0]])
assert len(token_lists) == 57
assert len(tag_lists) == 57

### Exercise 3.2
rubric={accuracy:1}

Now predict SRLabels for the test data and output your tags as we did for Lab 1, with a unique integer index in the first column and your predicted tags in the second. The file should have a header of `Id,Predicted`. A lot of your output will be `<null>`, but that's okay. Note again that you need to have run the cell from **3.1** before you call `prepare_dataset` so it uses your new version of `prepare_SRL_data` rather than the original one from **1.1*** which is only suitable for test data.

In [81]:
EPOCHS = 20 
model = biLSTMTagger(WORD_TO_IDX, TAG_TO_IDX)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [82]:
# Generate a small training set containing the 50 first batches
mini_trainset = list(train_dataloader)
print()

for epoch in range(EPOCHS):  
    print(f"Epoch {epoch+1}")
    tot_loss = 0    

    for sent_batch, tag_batch in mini_trainset:

        # `zero_gread()`
        model.zero_grad()
        
        output = model(sent_batch)
        output = output.permute(1,2,0) # batch size x 3 x sentence length
        loss = loss_function(output, tag_batch)
        loss.backward()
        tot_loss += loss
        optimizer.step()
        
    print(f"Avg loss per sentence: {tot_loss/len(mini_trainset)}")


Epoch 1
Avg loss per sentence: 0.594574511051178
Epoch 2
Avg loss per sentence: 0.3173092305660248
Epoch 3
Avg loss per sentence: 0.286975622177124
Epoch 4
Avg loss per sentence: 0.2703966796398163
Epoch 5
Avg loss per sentence: 0.2589440941810608
Epoch 6
Avg loss per sentence: 0.2498330920934677
Epoch 7
Avg loss per sentence: 0.24211423099040985
Epoch 8
Avg loss per sentence: 0.23557239770889282
Epoch 9
Avg loss per sentence: 0.2301451414823532
Epoch 10
Avg loss per sentence: 0.22567777335643768
Epoch 11
Avg loss per sentence: 0.2219036966562271
Epoch 12
Avg loss per sentence: 0.2187049239873886
Epoch 13
Avg loss per sentence: 0.2159619927406311
Epoch 14
Avg loss per sentence: 0.21358978748321533
Epoch 15
Avg loss per sentence: 0.21150198578834534
Epoch 16
Avg loss per sentence: 0.2096129208803177
Epoch 17
Avg loss per sentence: 0.20773008465766907
Epoch 18
Avg loss per sentence: 0.20606881380081177
Epoch 19
Avg loss per sentence: 0.20440028607845306
Epoch 20
Avg loss per sentence: 0

In [97]:
test_dataset = prepare_dataset(test_files) # must use the new prepare_SRL_data for this to work
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
all_predicted_tags, sys = [], []
#your code here

for sent, _ in test_dataloader:
    output = model(sent)
    # argmax of the reuslt of `model`
    tag = torch.argmax(output, dim=2).squeeze(1).permute(1,0)
    # for t in tag:
    #     sys.extend(t.tolist())
    # all_predicted_tags.extend([IDX_TO_TAG[i] for i in sys])
    all_predicted_tags.extend([IDX_TO_TAG[i] for j in tag for i in j.tolist()])


An assertion to check your code 

In [98]:
assert len(all_predicted_tags) == 540100

Write your predictions in the file `kaggle_tags.csv`.

In [100]:
HEADER = 'Id,Predicted'


import csv
with open('kaggle_tags.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Id','Predicted'])
    for id, i in enumerate(all_predicted_tags):
        writer.writerow([id,i])

Assertions to check that your output file contains the correct number of tokens and sentences.

In [101]:
with open('kaggle_tags.csv') as f:
    lines = f.readlines()
    assert len(lines) == 540101
    assert lines[0].strip() == HEADER
    assert lines[1].startswith("0,")
    assert lines[-1].startswith("540099,")    
print("Success!")

Success!


### Exercise 3.3 (optional)
rubric={accuracy:2}

When you've generated them, submit your tags to our kaggle competition. The competition page can be found [here](https://www.kaggle.com/t/da7da8c6cc0442c5bc27811a525530be). You'll first need to join our competition.

Your submission is evaluated in two sets: Public and Private. You will be able to see how you rank on the Public Leaderboard of the kaggle competition, which is evaluated on the Public data set. The final rankings, however, will be determined by your results on the Private data set.

Try to beat the baseline model (which was trained with the parameters above, but fewer epochs) in the Public leaderboard. You should be able to do this with your initial system you have built if you have correctly followed the instructions in this lab.

### Exercise 3.4 (optional)
rubric={raw:2}

Now compete to generate the best SRL tags. There's lots of room for improvement. You can build a deeper model, play around with hyperparameters, loss functions, regularization, etc. If you'd really like to push the limits, you can incorporate pre-trained word embeddings into the model (including contextualized word embeddings like BERT), add features from POS tagging, chunking or parsing (you would add them to the input tensors in the same way you did the predicate binary feature), or even add a CRF layer or some other kind of discrete optimization!

Important:  In order to provide a level playing field, you must train your models on *only* the training data.  Combining the training and development data into a single training set will disqualify you from the Kaggle.

As said, points in this exercise will be given based on ranking in the *Private* leaderboard at the deadline. 

- 1st: 2 points
- 2nd: 1.8 point
- 3rd: 1.6 point
- 4th: 1.4 point
- 5th: 1.2 point
- 6th: 1 point
- 7th: 0.8 point
- 8th 0.6 point
- 9th: 0.4 point 
- 10th: 0.2 point 

Again, make sure we can duplicate your result by running your code.